In [1]:
import sys


In [3]:
sys.argv.pop()

'-f'

In [4]:
sys.argv

['/Users/aaronciuffo/Virtualenvs/python2/lib/python2.7/site-packages/ipykernel/__main__.py']

In [5]:
from __future__ import print_function
import httplib2
import os

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage 
from apiclient import errors as gerror

SCOPES = 'https://www.googleapis.com/auth/drive'
# SCOPES = 'https://www.googleapis.com/auth/drive.file' # try this one!
# SCOPES = 'https://www.googleapis.com/auth/drive.metadata.readonly'
CLIENT_SECRET_FILE = 'client_secret_qs.json'
APPLICATION_NAME = 'Drive API Python Quickstart'

def getCredentials():
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        try:
            os.makedirs(credential_dir)
        except (IOError, OSError) as e:
            print(e)
        

    credential_path = os.path.join(credential_dir, 'drive-python.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = 'foo'
        credentials = tools.run_flow(flow, store, None)
    return(credentials)






In [6]:
credentials = getCredentials()
http = credentials.authorize(httplib2.Http())
service = discovery.build('drive', 'v3', credentials=credentials)

In [109]:
class gDrive():
    '''
    creates a google drive interface object
    
    Accepts:
    google drive v3 service object: (discover.build('drive', 'v3', credentials = credentials_object)
    
    '''
    def __init__(self, object):
        self.service = object
        # https://developers.google.com/drive/v3/web/mime-types
        self.mimeTypes = {'audio': 'application/vnd.google-apps.audio',
                          'docs': 'application/vnd.google-apps.document',
                          'drawing': 'application/vnd.google-apps.drawing',
                          'file': 'application/vnd.google-apps.file',
                          'folder': 'application/vnd.google-apps.folder',
                          'forms': 'application/vnd.google-apps.form',
                          'mymaps': 'application/vnd.google-apps.map',
                          'photos': 'application/vnd.google-apps.photo',
                          'slides': 'application/vnd.google-apps.presentation',
                          'scripts': 'application/vnd.google-apps.script',
                          'sites': 'application/vnd.google-apps.sites',
                          'sheets': 'application/vnd.google-apps.spreadsheet',
                          'video': 'application/vnd.google-apps.video'}
    
#     types = property()
    
    @property
    def types(self):
        print('supported mime types:')
        for key in self.mimeTypes:
            #print('%10s: %s' % (key, self.mimeTypes[key]))
            print('{:8} {val}'.format(key+':', val=self.mimeTypes[key]))
    
    def quote(self, string):
        '''
        add double quotes arounda string
        '''
        return('"'+str(string)+'"')
    
    def addFile(self, name = None, ftype = False, parents = None):
        body={}
        if name is None:
            print('specify a folder or file name')
            return(False)
        else:
            body['name'] = name
        
        if ftype in self.mimeTypes:
            body['mimeType'] = self.mimeTypes[ftype]
        
        if isinstance(parents, list):
            body['parents'] = parents
        elif parents:
            body['parents'] = [parents]
        
        print(body)
        result = self.service.files().create(body=body).execute()
        
        #body = {'name':'release the schmoo!', 'mimeType':'application/vnd.google-apps.folder', 'parents':["0BzC-V2QIsGRGWXNxNmhjc0FITDQ"]}
# service.files().create(body=body).execute()
        
        
        return(result)
    
    
    def search(self, name = None, trashed = False, ftype = False, fuzzy = False, date = None, parent = None):
        '''
        search for an item by name and other properties in google drive
        
        accepts:
            name (string): item name in google drive - required
            trashed (bool): item is not in trash - default False
            ftype = (string): item is one of the known mime types (gdrive.mimeTypes) - default None
            fuzzy = (bool): substring search of names in drive
            date = (RFC3339 date string): currently searches for files created after date string (YYYY-MM-DD)
            parent = (string): google drive file id string
            
        returns:
            list of file dict
        '''
        queryDict={}
        if name is None:
            print('specify a folder or file name')
            return(False)
        else:
            if fuzzy:
                fVal = ' contains '
            else:
                fVal = '='
            queryDict['name'] = 'name'+fVal+self.quote(name)
            
        if isinstance(trashed, bool):
            if trashed:
                tVal='trashed=True'
            else:
                tVal='trashed=False'
            queryDict['trashed'] = tVal
        
        if ftype in self.mimeTypes:
            queryDict['mimeType'] = 'mimeType='+self.quote(self.mimeTypes[ftype])
            
        if parent:
            queryDict['parents'] = self.quote(parent)+' in parents'
                
        
        if date:
            queryDict['modifiedTime'] = 'modifiedTime >'+self.quote(date)
        
        
        query = ' and '.join(queryDict.values())
        print(query)
        
        try:
            result = self.service.files().list(q=query).execute()
            return(result)
        except gerror.HttpError as e:
            print('File Not Found')
            return(False)
        
# #         try:
# #             print('trying')
# #             result = self.service.files().list(q=query).execute()
# #         except gerror.HttpError as e:
# #             print('File Not Found')
# #             return(False)        
    
    
#     def search(self, name = None, trashed = False, ftype = False, fuzzy = False, parent = None):
        
        
#         if name is None:
#             print ('specify a folder or file name')
#             return(False)
#         else:
#             # this needs to be surrounded by double quotes
#             name = self.quote(name)
                
#         # give a reasonable value if this is null
#         if parent:
#             parentVal = self.quote(parent)            
#         else:
#             parentVal = self.quote('null')
            
#         if fuzzy:
#             fuzzyVal = ' contains '
#         else:
#             fuzzyVal = '='
            
#         supportedFeatures = {'name':'name'+fuzzyVal+self.quote(eval(name)), 
#                              'trashed': 'trashed='+eval(self.quote(trashed)), 
#                              'parent': self.quote(eval(parentVal))+' in parents',
#                              }
        
#         # exclude these if they are set to none or false
#         nullFeatures =['parent']
        
#         queryList = []
#         for each in supportedFeatures:
#             if each in nullFeatures and not eval(each):
#                 pass
#             else:
#                 queryList.append(str(supportedFeatures[each])) #+ str(myDict[each]))

#         query = ' and '.join(queryList)        
#         print(query)
        
#         try:
#             result = self.service.files().list(q=query).execute()
#         except gerror.HttpError as e:
#             print('File Not Found')
#             return(False)
        
#         return(result)

In [110]:
myDrive = gDrive(service)
# print(myDrive.search(name = 'py', ftype='folder', fuzzy = True, date ="2017-03-15"))

In [101]:
body = {'name':'release the schmoo!', 'mimeType':'application/vnd.google-apps.folder', 'parents':["0BzC-V2QIsGRGWXNxNmhjc0FITDQ"]}
service.files().create(body=body).execute()

{u'id': u'0BzC-V2QIsGRGT00xSjJHNy1Td1k',
 u'kind': u'drive#file',
 u'mimeType': u'application/vnd.google-apps.folder',
 u'name': u'release the schmoo!'}

In [112]:
myDrive.addFile('bumblefuck', ftype = 'folder', parents = "0BzC-V2QIsGRGWXNxNmhjc0FITDQ")

{'mimeType': 'application/vnd.google-apps.folder', 'parents': ['0BzC-V2QIsGRGWXNxNmhjc0FITDQ'], 'name': 'bumblefuck'}


{u'id': u'0BzC-V2QIsGRGOWtrNVg1YnNCU0k',
 u'kind': u'drive#file',
 u'mimeType': u'application/vnd.google-apps.folder',
 u'name': u'bumblefuck'}

In [54]:
service.files().create(mimetype = "application/vnd.google-apps.folder", name = 'blarg').execute()

TypeError: Got an unexpected keyword argument "mimetype"

In [32]:
service.files().list(q='name contains "img" and modifiedTime > "2017-03-15"').execute()

{u'files': [{u'id': u'11FyFTl8ZdaT-n2TnSlte_V3HZyEjUtYFpg',
   u'kind': u'drive#file',
   u'mimeType': u'image/jpeg',
   u'name': u'IMG_20170315_175455.jpg'},
  {u'id': u'1KekuY9jqMtiIGe0R-RKMH4tyZJvPwfo-Pg',
   u'kind': u'drive#file',
   u'mimeType': u'image/jpeg',
   u'name': u'IMG_20170315_175453.jpg'},
  {u'id': u'1z4IOI4lvNE6p_T2XXjNfhrTQ-k15n3IPIg',
   u'kind': u'drive#file',
   u'mimeType': u'image/jpeg',
   u'name': u'IMG_20170315_175452.jpg'},
  {u'id': u'1zZYokEqxDUs2-XVqjA1u62MNujnDxs0iMA',
   u'kind': u'drive#file',
   u'mimeType': u'image/jpeg',
   u'name': u'IMG_20170315_175444.jpg'},
  {u'id': u'1xyBuulzKSJUAXq6WZnmzRvmU9cvSo0tzMQ',
   u'kind': u'drive#file',
   u'mimeType': u'image/jpeg',
   u'name': u'IMG_20170315_175442.jpg'},
  {u'id': u'1eLCPiejmvFCPBFhy0xczuC2PulZfql9sIg',
   u'kind': u'drive#file',
   u'mimeType': u'image/jpeg',
   u'name': u'IMG_20170315_175115.jpg'},
  {u'id': u'167eOpKSBFl9dkXpTgN3sy3N3giuaMpgYkA',
   u'kind': u'drive#file',
   u'mimeType': u'i

In [ ]:
service.files().list(q='name contains "Letters" and trashed=False and mimeType = "application/vnd.google-apps.folder"').execute()

In [ ]:
myDrive = gDrive(service)
myDrive.Xsearch("test", fuzzy = True)

In [ ]:
myDrive.types

In [ ]:
myDrive.search(name = "more cows")

In [ ]:
discovery.mimetypes